<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/discount_margin_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discount margin calculation

(Based on [two](http://quant.stackexchange.com/questions/8965/) [questions](https://quant.stackexchange.com/questions/37705/) by _Stack Exchange_ users HookahBoy and Kyle. Thanks!)

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql

In [ ]:
today = ql.Date(8, ql.October, 2014)
ql.Settings.instance().evaluationDate = today

#### The question

Given a floating-rate bond price, we want to find the corresponding discount margin.  This is one in a class of similar problems: we have a calculation which is not immediate to do directly, but is straightforward to do in the opposite direction; in this case, find the price of a bond when discounting its coupons at a spread over LIBOR.

The general idea is to implement the inverse calculation (DM to price) and then to use a solver to determine the correct input given the result. First, we build the bond.

In [ ]:
forecast_curve = ql.RelinkableYieldTermStructureHandle()
discount_curve = ql.RelinkableYieldTermStructureHandle()

In [ ]:
index = ql.Euribor6M(forecast_curve)

In [ ]:
issueDate = ql.Date(13,ql.October,2014)
maturityDate = ql.Date(13,ql.October,2024)

schedule = ql.Schedule(issueDate, maturityDate,
                       ql.Period(ql.Semiannual), ql.TARGET(),
                       ql.Following, ql.Following,
                       ql.DateGeneration.Backward, False)

In [ ]:
bond = ql.FloatingRateBond(settlementDays = 3,
                           faceAmount = 100,
                           schedule = schedule,
                           index = index,
                           paymentDayCounter = ql.Actual360(),
                           paymentConvention = ql.Following,
                           fixingDays = index.fixingDays(),
                           gearings = [],
                           spreads = [],
                           caps= [],
                           floors = [],
                           inArrears = False,
                           redemption = 100.0,
                           issueDate = issueDate)
bond.setPricingEngine(ql.DiscountingBondEngine(discount_curve))

Now we link the forecast curve to the current Euribor curve (whatever that is; I'm using a flat one as an example, but it could as well be a real one)...

In [ ]:
forecast_curve.linkTo(ql.FlatForward(0, ql.TARGET(), 0.002, ql.Actual360()))

...and the discount curve to the Euribor curve plus the discount margin.

In [ ]:
DM = ql.SimpleQuote(0.0)
discount_curve.linkTo(ql.ZeroSpreadedTermStructure(forecast_curve,
                                                   ql.QuoteHandle(DM)))

Setting a value to the DM quote will affect the bond price: this gives us the knob to manipulate in order to find the solution of our problem.

In [ ]:
print(bond.cleanPrice())

100.00000000000001


In [ ]:
DM.setValue(0.001)
print(bond.cleanPrice())

98.99979030764418


To invert the calculation, we encapsulate the above into a function.  The Python language makes it easier to write it in a general way; the function below takes the target price, and returns another function that takes a value for the discount margin and returns the difference between the corresponding price and the target.  In C++, we would create a function object taking the target price in its constructor and returning the difference from its `operator()`.

In [ ]:
def F(price):
    def _f(s):
        DM.setValue(s)
        return bond.cleanPrice() - price
    return _f

In [ ]:
f = F(98.9997903076)
print(f(0.0))
print(f(0.002))

1.00020969240002
-0.9901429992548856


We want to find the value of the discount margin that causes the calculated price to equal the target price, that is, that causes the error to be 0; and for that, we can use a solver.

In [ ]:
margin = ql.Brent().solve(F(99.6), 1e-8, 0.0, 1e-4)
print(margin)

0.00039870328652332745


We can verify that this works by setting the margin to the returned value and checking that the bond price equals the input:

In [ ]:
DM.setValue(margin)
print(bond.cleanPrice())

99.59999988275108


However, note that the spread above is continuously compounded. You might want to see the discount margin in the same units as the index fixings:

In [ ]:
value_date = index.valueDate(today)
maturity_date = index.maturityDate(value_date)
print(ql.InterestRate(margin, discount_curve.dayCounter(),
                      ql.Continuous, ql.NoFrequency)
      .equivalentRate(index.dayCounter(),
                      ql.Simple, index.tenor().frequency(),
                      value_date, maturity_date))

0.039874 % Actual/360 simple compounding


#### Not just for bonds

The approach I described can be generalized to any problem in this class. Here I'll use it to get the implied volatility of an Asian option: first I'll create the instrument...

In [ ]:
exerciseDate = today + ql.Period(1,ql.Years)
fixingDates = [ today + ql.Period(n,ql.Months) for n in range(1,12) ]
option = ql.DiscreteAveragingAsianOption(
    ql.Average.Arithmetic,
    0.0, 0,
    fixingDates,
    ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
    ql.EuropeanExercise(exerciseDate))

...and an engine, taking care of writing the input volatility as a quote.

In [ ]:
sigma = ql.SimpleQuote(0.20)

riskFreeCurve = ql.FlatForward(0, ql.TARGET(), 0.01, ql.Actual360())
volatility = ql.BlackConstantVol(0, ql.TARGET(),
                                 ql.QuoteHandle(sigma), ql.Actual360())

process = ql.BlackScholesProcess(ql.QuoteHandle(ql.SimpleQuote(100.0)),
                                 ql.YieldTermStructureHandle(riskFreeCurve),
                                 ql.BlackVolTermStructureHandle(volatility))

In [ ]:
option.setPricingEngine(
    ql.MCDiscreteArithmeticAPEngine(process, "pseudorandom",
                                    requiredSamples=1000,
                                    seed=42))

Now we can use the same technique as above: the function below takes a target price and returns a function from the volatility to the pricing error:

In [ ]:
def F(price):
    def _f(v):
        sigma.setValue(v)
        return option.NPV() - price
    return _f

Using a solver, we can invert it to solve for any price:

In [ ]:
print(ql.Brent().solve(F(5.0), 1e-8, 0.20, 1e-4))

0.20081193864526342


In [ ]:
print(ql.Brent().solve(F(6.0), 1e-8, 0.20, 1e-4))

0.24362397543255393
